# Measuring hydrostatic masses for SDSSRM-XCS galaxy clusters

In this notebook we perform new measurements of hydrostatic mass for a subset of the SDSSRM-XCS volume-limited, temperature error limited, galaxy cluster sample. We also aim to use the masses to construct mass observable relations between mass and various X-ray quantities, as well as mass and the optical quantity richness. These mass measurements are meant as a companion to the [Giles et al. (2022)](https://arxiv.org/abs/2202.11107) paper, which constructed the sample and reported measurements of temperature and luminosity.

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os
from tqdm import tqdm
from warnings import warn

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import xcs3p_colour, xcs_cosmo, direct_comparison_plot, find_lims, fit_comp

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 100
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sourcetools.mass import inv_abel_dens_onion_temp
from xga.relations.fit import scaling_relation_odr
from xga.relations.clusters.MT import arnaud_m500
from xga.models.misc import power_law
from xga.exceptions import NoProductAvailableError, ModelNotAssociatedError
from xga.xspec import single_temp_apec


## Reading in sample file and Declaring XGA ClusterSample

Reading in the sample csv file, containing XAPA positions, redshifts, XCS3P $R_{500}$ estimates, and richnesses. This is used to define the XGA ClusterSample, which in turn is what hydrostatic mass measurements are run on. We make sure to turn peak-finding off, as we base our measurements on XAPA positions.

In [3]:
xcs3p = pd.read_csv("../../sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")
xcs3p_r2500 = pd.read_csv("../../sample_files/xcs3p_sdssrm_r2500_props_nonvol_lim_temperr_100%_clusters.csv")

In [5]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xcs3p['xapa_ra'].values
dec = xcs3p['xapa_dec'].values
z = xcs3p['z'].values
# Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
n = xcs3p['name'].values
# In arcminutes, ClusterSample declaration will convert to kpc using the provided cosmology
r500 = Quantity(xcs3p['r500'].values, 'arcmin')
# Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
r = xcs3p['richness'].values
r_err = xcs3p['richness_err'].values

# Declaring the actual ClusterSample instance for the XCS sample
xcs_srcs = ClusterSample(ra, dec, z, n, r500=r500, richness=r, richness_err=r_err, cosmology=xcs_cosmo, 
                         load_fits=True, use_peak=False, clean_obs=True, clean_obs_reg='r500', 
                         clean_obs_threshold=0.7)

Declaring BaseSource Sample:  31%|█████████████▏                             | 46/150 [02:58<06:06,  3.52s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/samples/base.py:90: UserWarning: Source SDSSXCS-1884 does not appear to have any XMM data, and will not be included in the sample.
  warn("Source {n} does not appear to have any XMM data, and will not be included in the "
Declaring BaseSource Sample:  76%|███████████████████████████████▉          | 114/150 [07:23<02:19,  3.88s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/samples/base.py:90: UserWarning: Source SDSSXCS-210 does not appear to have any XMM data, and will not be included in the sample.
  warn("Source {n} does not appear to have any XMM data, and will not be included in the "
Setting up Galaxy Clusters:   7%|██▉                                         | 10/148 [00:26<05:51,  2.55s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:1039: UserWarning: A previous annular spectra profile fit for SDSSXCS-2346 w

Setting up Galaxy Clusters:  39%|████████████████▉                           | 57/148 [04:51<09:45,  6.43s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-8060 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  42%|██████████████████▍                         | 62/148 [05:12<06:29,  4.53s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-1628 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  45%|███████████████████▌                        | 66/148 [05:36<08:18,  6.07s/it]/mnt/p

Setting up Galaxy Clusters:  67%|█████████████████████████████▍              | 99/148 [09:21<02:35,  3.17s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-457 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  68%|█████████████████████████████              | 100/148 [09:26<02:56,  3.68s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-35 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  71%|██████████████████████████████▌            | 105/148 [09:57<05:34,  7.78s/it]/mnt/pact

In [6]:
# This cluster is causing some problems for an unknown reason, but will be restored later when I've figured out why
del xcs_srcs['SDSSXCS-408']

## Hydrostatic Mass Measurement

Here we measure hydrostatic mass profiles for clusters in the SDSSRM-XCS sample, using $S_{B}$ profiles to calculate density profiles and combining them with temperature profiles to measure hydrostatic mass. 

We fit double beta models to the PSF-corrected $S_B$ profiles, from which we infer density profiles and fit simplified Vikhlinin density models to them. Temperature profiles are measured from annular spectra, deprojected, and then fitted with simplified Vikhlinin temperature models.

In this instance we choose to generate profiles out to the $R_{500}$ value measured by Giles et al. 2022. Each annulus is required to have a total of 1500 counts in each annulus when the annulus is applied to the combined observations.

In [11]:
sb_dblbeta_simpvikh_1_totmc1500_mp = inv_abel_dens_onion_temp(xcs_srcs, xcs_srcs.r500, 'double_beta', 
                                                                'simple_vikhlinin_dens', 'simple_vikhlinin_temp', 
                                                                'r500', temp_annulus_method='min_cnt', 
                                                                temp_min_cnt=Quantity(1500, 'ct'), 
                                                                temp_use_combined=True)

Running XSPEC Fits: 100%|██████████████████████████████████████████████████████| 2/2 [09:53<00:00, 296.92s/it]
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/mass.py:51: UserWarning: The global temperature fit for SDSSXCS-21847 has failed, and as such we're very unlikely to be able to measure a mass and we're not even going to try.
  warn("The global temperature fit for {} has failed, and as such we're very unlikely to be able to measure "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/mass.py:51: UserWarning: The global temperature fit for SDSSXCS-6307 has failed, and as such we're very unlikely to be able to measure a mass and we're not even going to try.
  warn("The global temperature fit for {} has failed, and as such we're very unlikely to be able to measure "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-290 cannot be created, the data quality is too low. As such a set of four annuli 

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-26424 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-23 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-617 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProje

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-11613 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-5939 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-11881 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmP

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:326: UserWarning: SDSSXCS-5977 annular spectra profile fit was not successful
  warnings.warn("{src} annular spectra profile fit was not successful".format(src=ann_spec.src_name))
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:326: UserWarning: SDSSXCS-8366 annular spectra profile fit was not successful
  warnings.warn("{src} annular spectra profile fit was not successful".format(src=ann_spec.src_name))
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:326: UserWarning: SDSSXCS-26424 annular spectra profile fit was not successful
  warnings.warn("{src} annular spectra profile fit was not successful".format(src=ann_spec.src_name))
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:326: UserWarning: SDSSXCS-617 annular spectra profile fit was not successful
  warnings.warn("{src} annular spectra profile fit was not successful".format(src=ann_spec.src_name))
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xsp

The chain is shorter than 50 times the integrated autocorrelation time for 6 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [ 690.87134067  633.23142544  875.74019839  809.14636477  888.81544967
 1013.19762009]
The chain is shorter than 50 times the integrated autocorrelation time for 5 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [598.23531012 344.92171083 595.03876752 545.96368243 639.90790407
 619.48109622]


...

The chain is shorter than 50 times the integrated autocorrelation time for 3 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [418.15479106 442.25420266 271.84714181 426.91351835 285.19589453
 244.2843048 ]


Generating SDSSXCS-147 hydrostatic mass profile:  99%|███████████████████▋| 145/147 [1:19:35<00:57, 28.58s/it]

The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [829.32033678 801.76264745 247.42117201 801.25201021 373.18035639
 411.7039616 ]
The chain is shorter than 50 times the integrated autocorrelation time for 3 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [260.93835199 649.05824467 651.4155336  471.16512448 297.2678794
 267.55082059]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [413.9907665  593.01958219 766.50278312 326.55591456 141.96989802
 717.76636364]


Complete: 100%|███████████████████████████████████████████████████████████| 147/147 [1:21:07<00:00, 33.11s/it]

The chain is shorter than 50 times the integrated autocorrelation time for 6 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [1284.17264412 1480.04750649 1147.74378073 1315.70589272 1227.74571067
 1206.70123932]


In [12]:
# print("There are", len([m for m in sb_dblbeta_simpvikh_11_m500[:, 0] if np.isfinite(m)]), 
#       "clusters with successful mass measurements")

In [15]:
sb_dblbeta_simpvikh_1_mc1500_m500 = []

with tqdm(desc='Measuring Masses for SDSSRM-XCS Clusters', total=len(sb_dblbeta_simpvikh_1_totmc1500_mp)) as onwards:
    for mp in sb_dblbeta_simpvikh_1_totmc1500_mp:
        if mp is not None:
            rel_src = xcs_srcs[mp.src_name]
            m = mp.mass(rel_src.r500)[0]
        else:
            m = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
            
        sb_dblbeta_simpvikh_1_mc1500_m500.append(m)
        
        onwards.update(1)

sb_dblbeta_simpvikh_1_mc1500_m500 = Quantity(sb_dblbeta_simpvikh_1_mc1500_m500)


Measuring Masses for SDSSRM-XCS Clusters: 100%|████████████████████████████| 147/147 [00:01<00:00, 111.63it/s]


In [17]:
all_info = []
for src_ind, src in enumerate(xcs_srcs):
    mass = sb_dblbeta_simpvikh_1_mc1500_m500[src_ind, :].value
    
    if sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind] is not None:
        try:
            alt_r500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].overdensity_radius(500, src.redshift, src.cosmo)
            alt_m500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].mass(alt_r500)[0]
            
            alt_r2500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].overdensity_radius(2500, src.redshift, src.cosmo)
            alt_m2500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].mass(alt_r2500)[0]
        except ValueError:
            alt_m500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
            alt_m2500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
    else:
        alt_m500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
        alt_m2500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
                
    rich = src.richness.value
    z = src.redshift
    try:
        tx = src.get_temperature(src.r500).value
        lx = src.get_luminosities(src.r500, lo_en=Quantity(0.5, 'keV'), hi_en=Quantity(2.0, 'keV')).value
        lxce = src.get_luminosities(src.r500, inner_radius=src.r500*0.15, lo_en=Quantity(0.5, 'keV'), 
                                    hi_en=Quantity(2.0, 'keV')).value
    except ModelNotAssociatedError:
        tx = np.array([np.NaN, np.NaN, np.NaN])
        lx = np.array([np.NaN, np.NaN, np.NaN])
        lxce = np.array([np.NaN, np.NaN, np.NaN])
        
    all_info.append([z, *mass, *rich, *tx, *lx, *lxce, *alt_m500.value, *alt_m2500.value])

all_info = np.array(all_info)
all_info_df = pd.DataFrame(data=all_info, columns=['z', 'm500', 'm500-', 'm500+', 'richness', 'richness_err', 
                                                   'Tx500', 'Tx500-', 'Tx500+', 'Lx500', 'Lx500-', 'Lx500+', 
                                                   'Lxce500', 'Lxce500-', 'Lxce500+', 'alt_m500', 'alt_m500-', 
                                                   'alt_m500+', 'alt_m2500', 'alt_m2500-', 'alt_m2500+'])
all_info_df.to_csv('prelim_masses_xapa_centres.csv', index=False)

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/products/profile.py:1811: UserWarning: Some radii are outside the data range covered by the temperature or density profiles, as such you will be extrapolating based on the model fits.
  warn("Some radii are outside the data range covered by the temperature or density profiles, as such "
